In [ ]:
import os
import re
import spacy
import pandas as pd

# import requirements for converting the dataframe to Spacy Docs
from collections import defaultdict
from unicodedata import normalize
import regex
import random

# Load and process data from Coda

In [ ]:
FILE_PATH = "../assets/NER_assets/Ancient_Words.csv"
# read csv file
df = pd.read_csv(FILE_PATH)
# rename columns to fit code
df.rename(columns = {'Word':'Keyword', 'Category Types':'Label'}, inplace = True)
# If a cell is empty (NaN), Fill it with the value in its parallel "Early" column
for row in df:
    df['Quote'].fillna(df['Early Quote'], inplace=True)
    df['Word Before'].fillna(df['Early Word Before'], inplace=True)
    df['Word After'].fillna(df['Early Word After'], inplace=True)
    df['Label'].fillna(df['Early Category Type'], inplace=True)
# remove rows with no Keyword
df = df.dropna(subset=['Keyword'])
# Remove any row that isn't Greek
pat = '[ء-ي]+'
#df.Keyword.str.contains(pat)
df = df[~df.Keyword.str.contains(pat, na=False)]
#replace new line in df column
df['Keyword'].replace('\n', '', regex=True, inplace=True)
#replace numbers in df
df.replace('\d+', '', regex=True, inplace=True)
#replace hyphens in df column
df.replace('-', '', regex=True, inplace=True)
# replace comma in df column
df['Keyword'].replace(',', '', regex=True, inplace=True)
#replace period in df column
df['Keyword'].replace('\.', '', regex=True, inplace=True)
#replace interpunkt in df column
df['Keyword'].replace('\·', '', regex=True, inplace=True)
# replace multiple spaces in df column
df.replace(' +', ' ', regex=True, inplace=True)
# replace end punctuation in df column
df['Keyword'].replace('\s+$', '', regex=True, inplace=True)

df.fillna(0)
df.reset_index(drop=True, inplace=True)


In [ ]:
df

In [ ]:
# normalize table
FORMAT = 'NFKC'
for col in ['Keyword', 'Quote', 'Word Before', 'Word After']:
    df[col] = df[col].apply(lambda x: normalize(FORMAT, str(x)) if not isinstance(x, float) else x)

## Fix similar sentences

In [ ]:
df_grouped = pd.DataFrame(columns=['Quote', 'entities'])

# group all similar sentences together, and for each one append to entities: [label, word, word before, word after]
entities_list = []
for name, group in df.groupby('Quote'):
    entities = [
        [row['Label'], row["Keyword"], row['Word Before'], row['Word After']]
        for i, row in group.iterrows()
    ]
    # append to entities_list
    entities_list.append({'Quote': name, 'entities': entities})

# create new DataFrame from entities_list
df_grouped = pd.DataFrame(entities_list)

# sample random from df_grouped
df_grouped

In [ ]:
# after naively grouping similar sentences, we need to check if they are actually similar
# we do this by using regex with difference up to 10 chars
checked = []
similar_indices = []

for i, row in df_grouped.iterrows():
    # if we haven't checked this sentence yet
    if i not in checked:
        checked.append(i)
        similar_indices.append([i])
        # loop over all other sentences
        pattern = fr"{re.escape(row['Quote'])}"
        # add fuzzy matching with up to 10 mistakes
        pattern = fr"(?:{pattern}){{e<=10}}"

        for j, row2 in df_grouped.iterrows():
                # if the sentences are similar
            if regex.search(row['Quote'], row2['Quote']):
                if j not in checked:
                    # append to similar_indices
                    similar_indices[-1].append(j)
                    # append to checked
                    checked.append(j)

In [ ]:
print (similar_indices)
# choose a random group from similar_indices where the length of the group is more than 1
group = random.choice([group for group in similar_indices if len(group) > 1])
print (group)
# print the sentences in the group
for index in group:
    print (df_grouped.loc[index, 'Quote'])
    # print entities of each sentence in the group
    for entity in df_grouped.loc[index, 'entities']:
        print (entity)

In [ ]:
# find the longest sentence in each group of similar sentences
df_grouped_regexed = pd.DataFrame(columns=['Quote', 'entities'])
for indices in similar_indices:
    # if there is only one sentence in the group add it to df_grouped_regexed
    if len(indices) == 1:
        df_grouped_regexed = pd.concat([df_grouped_regexed, df_grouped.loc[indices[0]].to_frame().T], ignore_index=True)
        continue
    # find the longest sentence in the group
    longest = max(indices, key=lambda x: len(df_grouped.loc[x, 'Quote']))
    entities = []
    # concatenate all entities from the other sentences in the group
    for index in indices:
        entities.extend(df_grouped.loc[index, 'entities'])
    # create a new row with the longest sentence and the concatenated entities
    new_row = {'Quote': df_grouped.loc[longest, 'Quote'], 'entities': entities}
    # add the new row to df_grouped_regexed
    df_grouped_regexed = pd.concat([df_grouped_regexed, pd.DataFrame([new_row])], ignore_index=True)

In [ ]:
# sample random from df_grouped_regexed
df_grouped_regexed.sample(10)


In [ ]:
len(df_grouped)

In [ ]:
df_grouped=df_grouped_regexed

In [ ]:
# sample random from df_grouped
df_grouped.sample(10)
len(df_grouped)

## Fix similar words

In [ ]:
d = {ord('\N{COMBINING ACUTE ACCENT}'):None, ord('\N{COMBINING COMMA ABOVE}'):None, ord('\N{COMBINING REVERSED COMMA ABOVE}'):None}

def find_word_index(sentence, word, word_before, word_after, clean_chars=True):
    """Find the index of a word in a sentence, it can appear multiple times but we return by word_before and word_after
    :returns start and end index of word in sentence"""


    # find the index of the word in the sentence (return index of beginning of word)
    word_index = sentence.find(word)

    # if "clean_chars" is True, remove all accents from the sentence
    if clean_chars: 
        # return all chars of word before: [),.,·] (meaning none of these chars will be included) so we get only data contained within the sentence
        for char in [")", ".", "·", ',']:
            if word_before.find(char) != -1:
                word_before = word_before[word_before.find(char)+1:]
                print ("word before: ", word_before)
            if word_after.find(char) != -1:
                word_after = word_after[:word_after.find(char)]
                print ("word after: ", word_after)
    else:
        #do nothing
        pass

    pattern = fr"{re.escape(word_before)}({(word)}){re.escape(word_after)}"
    # add fuzzy matching with up to 3 mistakes
    pattern = fr"(?:{pattern}){{e<=3}}"
    if match := regex.search(pattern, sentence):
        print ("match 3: ", match.span(1))
        return match.span(1)

    # try matching only with word_before
    pattern = fr"{re.escape(word_before)}({(word)})"
    # add fuzzy matching with up to 3 mistakes
    pattern = fr"(?:{pattern}){{e<=2}}"
    if match := regex.search(pattern, sentence):
        print ("match with before: ", match.span(1))
        return match.span(1)

    # try matching only with word_after
    pattern = fr"({(word)}){re.escape(word_after)}"
    # add fuzzy matching with up to 3 mistakes
    pattern = fr"(?:{pattern}){{e<=2}}"
    if match := regex.search(pattern, sentence):
        print ("match with after: ", match.span(1))
        return match.span(1)

    # if no match found, return None
    print ("\nNo match found for", word, "in", sentence, "\nwith word_before", word_before, "and word_after", word_after)
    return None, None


In [ ]:
def find_word_index_test(sentence, word, word_before, word_after):
    """Find the index of a word in a sentence, it can appear multiple times but we return by word_before and word_after
    :returns start and end index of word in sentence"""

    # Remove unwanted characters from word_before and word_after
    word_before = remove_unwanted_chars(word_before)
    word_after = remove_unwanted_chars(word_after)

    # Try matching with word_before and word_after
    pattern = fr"{re.escape(word_before)}({(word)}){re.escape(word_after)}"
    match = fuzzy_search(pattern, sentence, max_mistakes=3)
    if match:
        return match.span(1)

    # Try matching with word_before only
    pattern = fr"{re.escape(word_before)}({(word)})"
    match = fuzzy_search(pattern, sentence, max_mistakes=2)
    if match:
        return match.span(1)

    # Try matching with word_after only
    pattern = fr"({(word)}){re.escape(word_after)}"
    match = fuzzy_search(pattern, sentence, max_mistakes=2)
    if match:
        return match.span(1)

    # If no match found, return None
    print(f"No match found for '{word}' in '{sentence}' with word_before '{word_before}' and word_after '{word_after}'")
    return None, None

def remove_unwanted_chars(text):
    """Remove unwanted characters from a string"""
    cleaned_text = ""
    for char in [")", ".", "·", ",", "’", "ανωθεν"]:
        if char in text:
            print(f"Removing character '{char}' from text")
            text = text
    text = text
    return text

def fuzzy_search(pattern, text, max_mistakes):
    """Perform a fuzzy search for a regular expression pattern in a string"""
    pattern = fr"(?:{pattern}){{e<={max_mistakes}}}"
    return regex.search(pattern, text)

In [ ]:
nlp = spacy.load("grc_proiel_trf") # I use small model for speed but you should use trf (transformer) model for better results

In [ ]:
TRAIN_DATA = [] # list of tuples connecting sentences and all entities in that sentence


# create train data: loop over all quotes, and all entities in each quote, and find the index of the word in the sentence so now it entities was [[label, word before, word after], [label, word before, word after]] and now it is [[start, end, label], [start, end, label]]
not_found_list = []
not_found = 0
for i, row in df_grouped.iterrows():
    # loop over all entities
    #print (row['Quote'])
    new_entities = []
    print(row['Quote'])
    for entity in row['entities']:
        # find the index of the word in the sentence
        #print (entity)
        print (entity[1])
        start, end = find_word_index(row['Quote'], entity[1], entity[2], entity[3], clean_chars=True)
        print (start, end, entity[1])
        # append to TRAIN_DATA
        if start is None:
            not_found += 1
            print ("not found: ", start, end)
            # add word and sentence to not_found list
            not_found_list.append([entity[1], row['Quote']])
        else:
            new_entities.append([start, end, entity[0]])
    TRAIN_DATA.append((row['Quote'], {'entities': new_entities}))
    # print length of TRAIN_DATA and sum of all entities in TRAIN_DATA
    print (len(TRAIN_DATA), sum([len(x[1]['entities']) for x in TRAIN_DATA]))
    #print not_found words and their sentences
    print (not_found_list)
    



In [ ]:
TRAIN_DATA = [] # list of tuples connecting sentences and all entities in that sentence
TRAIN_DATA_TEST = [] # list of tuples connecting sentences and all entities in that sentence
TRAIN_DATA_TEST_CLEAN = [] # list of tuples connecting sentences and all entities in that sentence

# create train data: loop over all quotes, and all entities in each quote, and find the index of the word in the sentence so now it entities was [[label, word before, word after], [label, word before, word after]] and now it is [[start, end, label], [start, end, label]]
not_found_list = []
not_found = 0
not_found_test = 0
not_found_clean = 0
for i, row in df_grouped.iterrows():
    # loop over all entities
    new_entities = []
    new_entities_test = []
    new_entities_clean = []
    for entity in row['entities']:
        # find the index of the word in the sentence using find_word_index with clean_chars=False
        start, end = find_word_index(row['Quote'], entity[1], entity[2], entity[3], clean_chars=False)
        if start is None:
            not_found += 1
            # add word and sentence to not_found list
            not_found_list.append([entity[1], row['Quote'], 'find_word_index'])
        else:
            new_entities.append([start, end, entity[0]])
        # find the index of the word in the sentence using find_word_index_test
        start_test, end_test = find_word_index_test(row['Quote'], entity[1], entity[2], entity[3])
        if start_test is None:
            not_found_test += 1
            # add word and sentence to not_found list
            not_found_list.append([entity[1], row['Quote'], 'find_word_index_test'])
        else:
            new_entities_test.append([start_test, end_test, entity[0]])
        # find the index of the word in the sentence using find_word_index with clean_chars=True
        start_clean, end_clean = find_word_index(row['Quote'], entity[1], entity[2], entity[3], clean_chars=True)
        if start_clean is None:
            not_found_clean += 1
        else:
            new_entities_clean.append([start_clean, end_clean, entity[0]])
    TRAIN_DATA.append((row['Quote'], {'entities': new_entities}))
    TRAIN_DATA_TEST.append((row['Quote'], {'entities': new_entities_test}))
    TRAIN_DATA_TEST_CLEAN.append((row['Quote'], {'entities': new_entities_clean}))
    # print length of TRAIN_DATA and sum of all entities in TRAIN_DATA
    print(f"TRAIN_DATA length: {len(TRAIN_DATA)}, sum of all entities: {sum([len(x[1]['entities']) for x in TRAIN_DATA])}")
    print(f"TRAIN_DATA_TEST length: {len(TRAIN_DATA_TEST)}, sum of all entities: {sum([len(x[1]['entities']) for x in TRAIN_DATA_TEST])}")
    print(f"TRAIN_DATA_TEST_CLEAN length: {len(TRAIN_DATA_TEST_CLEAN)}, sum of all entities: {sum([len(x[1]['entities']) for x in TRAIN_DATA_TEST_CLEAN])}")
    # print not_found words and their sentences for all methods
    not_found_set = set([(x[0], x[1]) for x in not_found_list if x[2] == 'find_word_index']) - set([(x[0], x[1]) for x in not_found_list if x[2] == 'find_word_index_test']) - set([(x[0], x[1]) for x in not_found_list if x[2] == 'find_word_index_clean'])
    if not_found_set:
        print(f"Words not found by find_word_index but found by other methods: {not_found_set}")
    not_found_set_test = set([(x[0], x[1]) for x in not_found_list if x[2] == 'find_word_index_test']) - set([(x[0], x[1]) for x in not_found_list if x[2] == 'find_word_index']) - set([(x[0], x[1]) for x in not_found_list if x[2] == 'find_word_index_clean'])
    if not_found_set_test:
        print(f"Words not found by find_word_index_test but found by other methods: {not_found_set_test}")
    not_found_set_clean = set([(x[0], x[1]) for x in not_found_list if x[2] == 'find_word_index_clean']) - set([(x[0], x[1]) for x in not_found_list if x[2] == 'find_word_index']) - set([(x[0], x[1]) for x in not_found_list if x[2] == 'find_word_index_test'])
    if not_found_set_clean:
        print(f"Words not found by find_word_index with clean_chars=True but found by other methods: {not_found_set_clean}")
    # print good results for all methods
    print(f"Good results for find_word_index: {len(new_entities)}")
    print(f"Good results for find_word_index_test: {len(new_entities_test)}")
    print(f"Good results for find_word_index with clean_chars=True: {len(new_entities_clean)}")
    
print(f"Words not found by any method: {not_found}")
print(f"Words not found by find_word_index: {not_found}")
print(f"Words not found by find_word_index_test: {not_found_test}")
print(f"Words not found by find_word_index with clean_chars=True: {not_found_clean}")

In [ ]:
not_found_list

In [ ]:
# Print TRAIN_DATA row where column value is a specific value
for row in TRAIN_DATA:
    #if " Ἐξήρτηται δ' ἐκ τῆς μεγάλης φλεβὸς καὶ τῆς ἀορτῆς, καὶ δι' αὐτοῦ φλέβες πολλαὶ καὶ πυκναί, κατατείνουσαι πρὸς τὴν τῶν ἐντέρων θέσιν, ἄνωθεν ἀρξάμεναι μέχρι κάτω" in row[0]:
    if " Ἔχει δὲ διαφορὰς πολλάς, καθάπερ ἡ κοιλία, καὶ τοῦτο τὸ μόριον" in row[0]:
        print (row)
        #print characters in index locations
        print (row[0][row[1]['entities'][0][0]:row[1]['entities'][0][1]])

In [ ]:
# print chracters in index location in a text row. the text is " Ἐξήρτηται δ' ἐκ τῆς μεγάλης φλεβὸς καὶ τῆς ἀορτῆς, καὶ δι' αὐτοῦ φλέβες πολλαὶ καὶ πυκναί, κατατείνουσαι πρὸς τὴν τῶν ἐντέρων θέσιν, ἄνωθεν ἀρξάμεναι μέχρι κάτω"ArithmeticError
for row in TRAIN_DATA:
    if "Ἐξήρτηται δ' ἐκ τῆς μεγάλης φλεβὸς καὶ τῆς ἀορτῆς, καὶ δι' αὐτοῦ φλέβες πολλαὶ καὶ πυκναί, κατατείνουσαι πρὸς τὴν τῶν ἐντέρων θέσιν, ἄνωθεν ἀρξάμεναι μέχρι κάτω" in row[0]:
    #if "Φυσωμένης δὲ τῆς ἀρτηρίας ἐν ἐνίοις μὲν οὐ κατάδηλον ποιεῖ, ἐν δὲ τοῖς μείζοσι τῶν ζῴων δῆλον ὅτι εἰσέρχεται τὸ πνεῦμα εἰς αὐτήν." in row[0]:
        print (row[0][row[1]['entities'][0][0]:row[1]['entities'][0][1]])
        # print all entities in the row and the characters in the index locations
        for entity in row[1]['entities']:
            print (entity, row[0][entity[0]:entity[1]])
        # print how many entities in the row
        print (len(row[1]['entities']))           

In [ ]:
def remove_unwanted_chars(text):
    """Remove unwanted characters from a string"""
    cleaned_text = ""
    for char in [")", ".", "·", ",", "’"]:
        if char in text:
            print(f"Removing character '{char}' from text")
            text = text.replace(char, "")
    return text.strip()

In [ ]:
text = "This is a test, ανωθεν."
cleaned_text = remove_unwanted_chars(text)
print(f"Input text: '{text}'")
print(f"Cleaned text: '{cleaned_text}'")

In [ ]:
# sample random from TRAIN_DATA
random.sample(TRAIN_DATA, 10)

# Add annotations from INCEpTION

In [ ]:
# extract all files in inception folder to temp folder
from cassis import *
import zipfile
import tempfile
import os
from spacy.training import Example



In [ ]:
def normalize_inception_tags(sentence, begin, end, label):
    """Problem: INCEpTION texts are in different formats, we want a uniform format defined by the constant FORMAT"""
    """When we normalize a sentence we lose the correct offsets of the entities, so we need to fix them"""
    """We use the above function find_word_index to find the correct offsets"""

    word = sentence[begin:end]
    # word before is the word before the entity (take 5 chars before if exists, else take maximum available)
    word_before = sentence[max(0, begin-5):begin]
    # word after is the word after the entity (take 5 chars after if exists, else take maximum available)
    word_after = sentence[end:min(len(sentence), end+5)]

    # normalize all variables
    sentence = normalize(FORMAT, sentence)
    print ("sentence: ", sentence)
    word = normalize(FORMAT, word)
    print ("word: ", word)
    word_before = normalize(FORMAT, word_before)
    print ("word before: ", word_before)
    word_after = normalize(FORMAT, word_after)
    print ("word after: ", word_after)

    return find_word_index(sentence, word, word_before, word_after)

In [ ]:
INCEPTION_TRAIN_DATA = []
with tempfile.TemporaryDirectory() as tempdir:
    for f in os.listdir("../assets/NER_assets/INCEpTION_files/test/"):
        if f.endswith(".zip"):
            with zipfile.ZipFile(os.path.join("../assets/NER_assets/INCEpTION_files/test/", f), 'r') as zip_ref:
                zip_ref.extractall(tempdir)
    # open typesystem and print content
    with open('{0}/{1}'.format(tempdir, "TypeSystem.xml"), 'rb') as f:
        typesystem = load_typesystem(f)

    # iterate over all files in temp folder
    for f in os.listdir(tempdir):
        # if file is a xmi file
        if f.endswith(".xmi"):
            # load xmi file
            with open(os.path.join(tempdir, f), 'rb') as f:
                print(f.name)
                # load typesystem from temp folder
                cas = load_cas_from_xmi(f, typesystem=typesystem)
                # get all entities from cas
                for sentence in cas.select(("de.tudarmstadt.ukp.dkpro.core.api.segmentation.type.Sentence")):
                    ents = []
                    for token in cas.select_covered('webanno.custom.CategoryType', sentence):
                        print(token.get_covered_text(), token.value("Value"))
                        # create a span with the token start and end and the label
                        # find begin and end position of the token relative to the sentence
                        start = token.begin - sentence.begin
                        end = token.end - sentence.begin

                        # normalize the sentence, word, word_before and word_after
                        begin, end = normalize_inception_tags(sentence.get_covered_text(), start, end, token.value("Value"))
                        ents.append((begin, end, token.value("Value")))
                    sentence = normalize(FORMAT, sentence.get_covered_text())
                    INCEPTION_TRAIN_DATA.append((sentence, {'entities': ents}))




In [ ]:
#print cas content
for token in cas.select_covered('webanno.custom.CategoryType', sentence):
    print(token.get_covered_text(), token.value("Value"))

In [ ]:
for t in INCEPTION_TRAIN_DATA:
    print (t[0]+ "\n")
    for entity in t[1]['entities']:
        print (t[0][entity[0]:entity[1]], entity[2])

In [ ]:
# sample random from INCEPTION_TRAIN_DATA and print for each entity text[start:end] and label
t = random.choice(INCEPTION_TRAIN_DATA)
print (t[0]+ "\n")
for entity in t[1]['entities']:
    print (t[0][entity[0]:entity[1]], entity[2])

In [ ]:
MERGED_TRAIN_DATA = TRAIN_DATA + INCEPTION_TRAIN_DATA

In [ ]:
len("Τελευταία γὰρ ἡ κύστις κεῖται, \r\nτὴν μὲν ἐξάρτησιν ἔχουσα τοῖς ἀπὸ τῶν νεφρῶν τεταμένοις \r\nπόροις παρὰ τὸν καυλὸν τὸν ἐπὶ τὴν οὐρήθραν τείνοντα, \r\nκαὶ σχεδὸν πάντῃ κύκλῳ λεπτοῖς καὶ ἰνώδεσιν ὑμενίοις \r\nἐστὶ προσειλημμένη, παραπλησίοις οὖσι τρόπον τινὰ τῷ \r\nδιαζώματι τοῦ θώρακος.1")

In [ ]:
docs = []
losses = {}
for text, annot in TRAIN_DATA:
    # train model
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in annot["entities"]:
        # first check char span by text[start:end] and compare with annot
        print (text[start:end], label)
        # if label isnt float NaN
        if label == label and label != None:
            span = doc.char_span(start, end, label=label, alignment_mode="expand")
            print (span)
            ents.append(span)
    # check overlapping entities (each spacy token should only be part of one entity)
    ents = spacy.util.filter_spans(ents)

    doc.ents = ents
    docs.append(doc)
    print(len(docs), sum([len(doc.ents) for doc in docs]))



In [ ]:
# check if charspan correctly found the entities, and all of them.
doc = random.choice(docs)

print ("Spacy entities, for line:")
print (doc.text)
print([(ent.text, ent.label_) for ent in doc.ents])

# locate doc in MERGED_TRAIN_DATA
line = None
for i, row in enumerate(MERGED_TRAIN_DATA):
    if row[0] == doc.text:
        line = i
        #print (row[1])
        break

print ("\nTable entities, for same line:")
for entity in MERGED_TRAIN_DATA[line][1]['entities']:
    print (MERGED_TRAIN_DATA[line][0][entity[0]:entity[1]], entity[2])

# check if there are entities in TRAIN_DATA that are not in spacy doc
for entity in MERGED_TRAIN_DATA[line][1]['entities']:
    if entity[2] not in [ent.label_ for ent in doc.ents]:
        print ("Entity not found in spacy doc: ", MERGED_TRAIN_DATA[line][0][entity[0]:entity[1]], entity[2])

In [ ]:
from spacy.tokens import DocBin

NER_DocBin = DocBin(docs=docs)
NER_DocBin.to_disk('../corpus/Merged_NER_dataset_{0}.spacy'.format(FORMAT))

In [ ]:
# split docs into train test and dev
from sklearn.model_selection import train_test_split
from pathlib import Path
from spacy.tokens import DocBin


train_docs, test_docs = train_test_split(docs, test_size=0.2)
train_docs, dev_docs = train_test_split(train_docs, test_size=0.2)

Path("../corpus/train").mkdir(parents=True, exist_ok=True)
Path("../corpus/dev").mkdir(parents=True, exist_ok=True)
Path("../corpus/test").mkdir(parents=True, exist_ok=True)

train = DocBin(docs=train_docs)
train.to_disk("../corpus/train/train_ner_{0}.spacy".format(FORMAT))
test = DocBin(docs=test_docs)
test.to_disk("../corpus/test/test_ner_{0}.spacy".format(FORMAT))
dev = DocBin(docs=dev_docs)
dev.to_disk("../corpus/dev/dev_ner_{0}.spacy".format(FORMAT))

print ("Train: ", len(train_docs))
print ("Test: ", len(test_docs))
print ("Dev: ", len(dev_docs))






## Equally distribute examples between train, dev and test data objects.
We need to properly represent all named entities in each set. We will use a stratification algorithm twice: 1st we split the data to train and test sets, and then we split the test set to dev and test. 

In [ ]:
import spacy
from spacy.tokens import Doc, DocBin, Span
nlp = spacy.load("grc_proiel_trf")

In [ ]:
FORMAT = 'NFKC'

In [ ]:
merged_docbin = DocBin().from_disk("../corpus/Merged_NER_dataset_{0}.spacy".format(FORMAT))
# get docs from new_docbin
merged_docbin_docs = list(merged_docbin.get_docs(nlp.vocab))

In [ ]:
# A method to split the multilabel data into two sets

from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit
from sklearn.utils import indexable, _safe_indexing
from sklearn.utils.validation import _num_samples
from sklearn.model_selection._split import _validate_shuffle_split
from itertools import chain

def multilabel_train_test_split(*arrays,
                                test_size=None,
                                train_size=None,
                                random_state=None,
                                shuffle=True,
                                stratify=None):
    """
    Train test split for multilabel classification. Uses the algorithm from: 
    'Sechidis K., Tsoumakas G., Vlahavas I. (2011) On the Stratification of Multi-Label Data'.
    """
    if stratify is None:
        return train_test_split(*arrays, test_size=test_size,train_size=train_size,
                                random_state=random_state, stratify=None, shuffle=shuffle)
    
    assert shuffle, "Stratified train/test split is not implemented for shuffle=False"
    
    n_arrays = len(arrays)
    arrays = indexable(*arrays)
    n_samples = _num_samples(arrays[0])
    n_train, n_test = _validate_shuffle_split(
        n_samples, test_size, train_size, default_test_size=0.25
    )
    cv = MultilabelStratifiedShuffleSplit(test_size=n_test, train_size=n_train, random_state=123)
    train, test = next(cv.split(X=arrays[0], y=stratify))

    return list(
        chain.from_iterable(
            (_safe_indexing(a, train), _safe_indexing(a, test)) for a in arrays
        )
    )

### 1st Split: train and test

In [ ]:
import numpy as np
# List of all labels as integers. 
# IMPORTANT: Make sure that the labels integers are indeed item[7].

labels_ints = [item[7] for item in merged_docbin.tokens for item in item]
#construct an ndarray with shape len(new_docbin), buffer is item[7] for item in new_docbin.tokens for item in item
labels_nd_array = np.ndarray(shape=(len(merged_docbin), len(np.unique(labels_ints))), buffer=np.array(labels_ints), dtype=int)
labels_nd_array.shape

In [ ]:
X_train, X_test = multilabel_train_test_split(merged_docbin_docs ,stratify=labels_nd_array, test_size=0.20)

In [ ]:
# X_train to X_train.spacy
X_train_docbin = DocBin(docs=X_train)
X_test_docbin = DocBin(docs=X_test)

In [ ]:
X_train_docbin

### 2nd split: dev and test

In [ ]:
# List of all labels as integers
X_labels_ints = [item[7] for item in X_test_docbin.tokens for item in item]
#construct an ndarray with shape len(new_docbin), buffer is item[7] for item in new_docbin.tokens for item in item
X_labels_nd_array = np.ndarray(shape=(len(X_test_docbin), len(np.unique(X_labels_ints))), buffer=np.array(X_labels_ints), dtype=int)
X_labels_nd_array.shape

In [ ]:
X_dev, X_test = multilabel_train_test_split(X_test ,stratify=X_labels_nd_array, test_size=0.50)

In [ ]:
X_dev_docbin = DocBin(docs=X_dev)

### Save new sets and validate the distribution

In [ ]:
# save the train and test data to disk
train_docbin = DocBin(docs=X_train)
train_docbin.to_disk("../corpus/train/ner_train/ner_train_{0}.spacy".format(FORMAT))
dev_docbin = DocBin(docs=X_dev)
dev_docbin.to_disk("../corpus/dev/ner_dev/ner_dev_{0}.spacy".format(FORMAT))
test_docbin = DocBin(docs=X_test)
test_docbin.to_disk("../corpus/test/ner_test/ner_test_{0}.spacy".format(FORMAT))


In [ ]:
train_data_labels = ([ent.label_ for docs in X_train for ent in docs.ents])
test_data_labels = ([ent.label_ for docs in X_test for ent in docs.ents])
dev_data_labels = ([ent.label_ for docs in X_dev for ent in docs.ents])


In [ ]:
# visualize the distribution of the labels in the train, test and dev data
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
sns.set(style="darkgrid")
plt.figure(figsize=(20,10))
train_data_labels = ([ent.label_ for docs in X_train for ent in docs.ents])
test_data_labels = ([ent.label_ for docs in X_test for ent in docs.ents])
dev_data_labels = ([ent.label_ for docs in X_dev for ent in docs.ents])
train_data_labels = pd.DataFrame(train_data_labels, columns=['labels'])
test_data_labels = pd.DataFrame(test_data_labels, columns=['labels'])
dev_data_labels = pd.DataFrame(dev_data_labels, columns=['labels'])
sns.countplot(data=train_data_labels, x='labels')
plt.title("Distribution of labels in the train data")
plt.figure(figsize=(20,10))
sns.countplot(data=test_data_labels, x='labels')
plt.title("Distribution of labels in the test data")
plt.figure(figsize=(20,10))
sns.countplot(data=dev_data_labels, x='labels')
plt.title("Distribution of labels in the dev data")

In [ ]:
# check distribution of classes in train and test set with respect to each other
import pandas as pd
train_data_labels = ([ent.label_ for docs in X_train for ent in docs.ents])
test_data_labels = ([ent.label_ for docs in X_test for ent in docs.ents])
dev_data_labels = ([ent.label_ for docs in X_dev for ent in docs.ents])
train_data_labels = pd.DataFrame(train_data_labels, columns=['labels'])
test_data_labels = pd.DataFrame(test_data_labels, columns=['labels'])
dev_data_labels = pd.DataFrame(dev_data_labels, columns=['labels'])
train_data_labels['dataset'] = 'train'
test_data_labels['dataset'] = 'test'
dev_data_labels['dataset'] = 'dev'
all_data_labels = pd.concat([train_data_labels, test_data_labels, dev_data_labels], ignore_index=True)
plt.figure(figsize=(20,10))
sns.countplot(data=all_data_labels, x='labels', hue='dataset')



In [ ]:
# visualize frequency of annotation labels in X_train set
train_data_labels = ([ent.label_ for docs in X_train for ent in docs.ents])
train_data_labels = pd.DataFrame(train_data_labels, columns=['labels'])
train_data_labels = train_data_labels['labels'].value_counts()
train_data_labels = pd.DataFrame(train_data_labels)
train_data_labels = train_data_labels.reset_index()
train_data_labels = train_data_labels.rename(columns={'index': 'label', 'labels': 'frequency'})
plt.figure(figsize=(20,10))
sns.barplot(data=train_data_labels, x='label', y='frequency')
plt.title("Frequency of labels in the train data")
plt.xticks(rotation=90)

In [ ]:
print ("test:", len(X_test), "dev:", len(X_dev), "train:", len(X_train))


In [ ]:
# show distribution in numbers
grouped_labels = all_data_labels.groupby(['labels', 'dataset']).size()
print(grouped_labels)